# Generating a FlipBook from Random Burst Images

CS 543 Final Project


In [17]:
import os
import random
import cv2
import shutil
import string
from tqdm import tqdm

In [18]:
input_data_directory = 'data'

files = os.listdir(input_data_directory)
video_files = [file for file in files if os.path.isfile(
    os.path.join(input_data_directory, file))]

video_filename = None
if video_files:
    video_filename = random.choice(video_files)
    print(f"Randomly selected file: {video_filename}")
else:
    raise ValueError("ERROR: No video files found in the directory.")

Randomly selected file: DJI_0876.MOV


In [19]:
temp_directory = "temp"

# Delete any pre-existing temp data
for dir_name in os.listdir(temp_directory):
    dir_path = os.path.join(temp_directory, dir_name)
    if os.path.isdir(dir_path):
        shutil.rmtree(dir_path)

In [20]:
def extract_random_frames(video_file, percentage):
    cap = cv2.VideoCapture(os.path.join(input_data_directory, video_file))
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_size = max(1, int(percentage * total_frames))

    frame_indices = sorted(random.sample(range(total_frames), sample_size))

    output_folder = os.path.join(temp_directory, video_file)
    os.makedirs(output_folder, exist_ok=True)

    frame_count = 0
    current_index = 1
    saved_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count in frame_indices:
            output_path = os.path.join(
                output_folder, "{:05}.jpg".format(current_index))
            cv2.imwrite(output_path, frame)
            saved_count += 1
            print(f"Got Image {current_index} / {len(frame_indices)}: {100 * (current_index / float(len(frame_indices))): .3f}%", end="\r", flush=True)
            current_index += 1

        frame_count += 1
        if saved_count >= sample_size:
            break

    cap.release()
    print(f"Extracted {saved_count} frames to '{output_folder}'.")
    return output_folder

sampling_percentage = 0.1
temp_data_folder = extract_random_frames(video_filename, sampling_percentage)

Extracted 125 frames to 'temp\DJI_0876.MOV'.


In [21]:
output_directory = "output"

def create_ground_truth_video_from_images(image_folder, output_video, framerate=30):
    images = sorted(
        [f for f in os.listdir(image_folder) if f.endswith(".jpg")],
        key=lambda x: int(os.path.splitext(x)[0])
    )

    if not images:
        print("No images found in the specified folder.")
        return

    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape
    size = (width, height)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, framerate, size)

    for image in images:
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        out.write(frame)

    out.release()
    print(f"Video created successfully: {output_video}")

create_ground_truth_video_from_images(temp_data_folder, os.path.join(output_directory, f"{os.path.splitext(video_filename)[0]}_gt.MP4"))

Video created successfully: output\DJI_0876_gt.MP4


In [22]:
def generate_random_name(length=10):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def rename_files_in_directory(directory):
    files = os.listdir(directory)
    original_num_files = len(files)
    
    jpg_files = [f for f in files if f.lower().endswith('.jpg')]
    
    used_names = set()

    for file in tqdm(jpg_files):
        while True:
            new_name = generate_random_name()
            if new_name not in used_names:
                used_names.add(new_name)
                break
        
        original_path = os.path.join(directory, file)
        new_path = os.path.join(directory, f"{new_name}.jpg")
        
        os.rename(original_path, new_path)
        # print(f"Renamed {file} to {new_name}.jpg")
    
    files = os.listdir(directory)
    new_num_files = len(files)
    if new_num_files == original_num_files:
        print(f"Original Num Images == New Num Images in {directory}")
    else:
        raise ValueError(f"Original Num Images != New Num Images in {directory}")
    
rename_files_in_directory(temp_data_folder)

100%|██████████| 125/125 [00:00<00:00, 246.25it/s]

Original Num Images == New Num Images in temp\DJI_0876.MOV
